# Ibis and the Python Ecosystem

Ibis strives to be a friendly component of the larger Python data science
ecosystem. In this notebook we'll walk through a few ways of using Ibis to
interact with other tools you may already be using.

## Loading In-Memory Data

Ibis doesn't only work with existing databases, it can also work with in-memory
data too.

To load in-memory data into `ibis` you can use `ibis.memtable`. This takes in
either an existing dataframe-like object (currently `pandas`, `polars`, and
`pyarrow` are supported), or a dict of column-name -> data to use.

In [ ]:
import ibis

ibis.options.interactive = True

In [ ]:
# Create a table from a pandas.DataFrame
import pandas as pd

items = ibis.memtable(
    pd.DataFrame(
        {
            "item": ["apple", "banana", "orange"],
            "price": [0.75, 0.23, 0.92],
            "avg_weight": [0.120, 0.118, 0.150],
        }
    )
)

items

In [ ]:
# Create a table from a pyarrow.Table
import pyarrow as pa

inventory = ibis.memtable(
    pa.table(
        {
            "item": ["apple", "banana", "orange"],
            "count": [212, 125, 90],
        }
    )
)

inventory

In [ ]:
# Create a table from a polars.DataFrame
import polars as pl

orders = ibis.memtable(
    pl.DataFrame(
        {
            "id": [1234, 1234, 1235, 1235, 1236],
            "item": ["apple", "orange", "banana", "orange", "banana"],
            "count": [5, 3, 4, 10, 6],
        }
    )
)

orders

These tables can then be queried the same as any other ibis table.

In [ ]:
# Find the most expensive item
items.order_by(ibis.desc("price")).limit(1)

In [ ]:
# Find the product with the most items in stock
inventory.order_by(ibis.desc("count")).limit(1)

In [ ]:
# Find the most expensive item per kg
items.mutate(price_per_kg=items.price / items.avg_weight).order_by(ibis.desc("price_per_kg")).limit(1)

You can even compose queries that make use of tables of different input types.
Here we compose a query that joins a `pandas.DataFrame` with a
`polars.DataFrame` (more on joins in a later notebook).

In [ ]:
# Compute the total cost of each order
order_totals = (
    orders.join(items, "item")                             # 1. Join orders with items by item name
    .group_by("id")                                        # 2. Group By the order id
    .agg(total=lambda t: (t["price"] * t["count"]).sum())  # 3. Compute the total cost as sum(price * count) per order
)

order_totals

## Aside: Ibis's magical `_` ("Deferred") API

Ibis's API is optimized to be able to construct complex queries through method
chaining. For example, in the above query we chain a `join`, `group_by` and
`agg` call together.

Looking closely though, you can see that when constructing the `total`
aggregate in the `agg` call we've passed in a `lambda` function. In this case,
that `lambda` takes the `ibis.Table` object output by the prior `group_by` call
and returns an aggregate value constructed from columns in that table. The
`lambda` lets us reference the current table in the chain directly, avoiding
the need to create an intermediate python variable.

Another way of spelling this (one some might consider nicer syntax) is through
the usage of ibis's `_` API. This is a "magical" object that constructs "deferred
expressions". Everywhere you see `_` in the expression you should read "the
current table (`self`) in the method being called".

For example, all of these calls do the same thing:

In [ ]:
from ibis import _

In [ ]:
# Explicitly referencing `items.price` and `items.avg_weight`
t1 = items.mutate(price_per_kg=items.price / items.avg_weight)

t1

In [ ]:
# Using a lambda function
t2 = items.mutate(price_per_kg=lambda t: t.price / t.avg_weight)

t2

In [ ]:
# Using deferred expressions (the `_` object)
t3 = items.mutate(price_per_kg=_.price / _.avg_weight)

t3

We can even check that they're the same by asserting that the expressions
backing each `ibis.Table` are identical (remember, `ibis` uses a lazy
expression API behind the scenes):

In [ ]:
assert t1.equals(t2)  # assert the expressions backing t1 and t2 are equivalent
assert t2.equals(t3)  # assert the expressions backing t2 and t3 are equivalent

### Exercise 1: Using Deferred Expressions

Putting this all together, we can now rewrite the `join -> group_by -> agg`
operation from above using a deferred expression.

Your job is to rewrite `order_totals` (duplicated below) using a deferred
expression instead of a `lambda` function:

In [ ]:
# Compute the total cost of each order
order_totals = (
    orders.join(items, "item")
    .group_by("id")
    .agg(total=lambda t: (t["price"] * t["count"]).sum())  # <- use `_` here instead of a `lambda` function
)

order_totals

Note that you don't have to use deferred expressions if you don't want to!
Queries using deferred expressions may always be expressed without them (using
either a `lambda` function, or an intermediate python variable) - they're there
as friendly "syntactic sugar" to ease writing complex queries when you need
them.

#### Solution

In [ ]:
%load solutions/nb02_ex01.py

## Mixing memtables with database tables

When querying a `memtable`, `ibis` will use the current default backend
(`duckdb` by default), _unless_ the query also makes use of a table attached to
some other database. This means that memtables may be used as efficient lookup
tables when composing queries against external systems.

Here we load the same `penguins` dataset as in notebook 1:

In [ ]:
con = ibis.duckdb.connect("palmer_penguins.ddb", read_only=True)
penguins = con.table("penguins")

penguins

In [ ]:
coordinates = ibis.memtable(
    {
        "island": ["Biscoe", "Dream", "Torgersen"],
        "latitude": [-65.433333, -64.733333, -64.766667],
        "longitude": [-65.5, -64.233333, -64.083333],
    }
)

coordinates

In [ ]:
penguins.join(coordinates, "island")

### Exercise 2: Add a Column for Scientific Name

Like all species, the penguins here have scientific names. These are available
in the `penguin_species.jsonl` file in the tutorial repo.

In [ ]:
!cat penguin_species.jsonl

Ibis currently doesn't have a `read_json` method. No worries though, since this
dataset is small we can read the data using a different tool and pass it to
`ibis` as a memtable.

Your job is to:
- Read in the `penguin_species.jsonl` file. You might find the
  `pandas.read_json` function useful (note you'll need to pass in `lines=True`)
- Coerce it to a `memtable`.
- Join the original `penguins` table with the new `species` memtable to label
  every row with its proper scientific name.

#### Solution

In [ ]:
%load solutions/nb02_ex02.py

## Exporting In-memory Data

Similar to how `ibis` can consume multiple in-memory data formats, we also can
return queries in multiple input formats using one of the various `to_*`
methods:

- `query.to_pandas()`: returns results as a `pandas.DataFrame`
- `query.to_polars()`: returns results as a `polars.DataFrame`
- `query.to_pyarrow()`: returns results as a `pyarrow.Table`
- `query.to_pyarrow_batches()`: returns results as a `pyarrow.RecordBatchReader`
- `query.to_torch()`: returns results as a `dict` of `torch.Tensor`s

Supporting multiple output formats lets you efficiently compose ibis with
existing systems. Have some downstream code that expects a `pandas.DataFrame`?
Use ibis to do some initial processing then hand off the result to your
existing code. Our wide support for popular Python data tools means `ibis` can
often fit nicely in to existing workflows without forcing all code be ported to
use `ibis`.

In [ ]:
mean_mass = penguins.group_by("species").body_mass_g.mean()
mean_mass

In [ ]:
# Get results as a pandas DataFrame
df = mean_mass.to_pandas()

print(type(df))

df

In [ ]:
# Get results as a polars DataFrame
mean_mass.to_polars()

In [ ]:
# Get results as a pyarrow Table
mean_mass.to_pyarrow()

## Integrating Ibis with Downstream Tools

Some downstream tools may be able to accept `ibis.Table` objects directly as
inputs without requiring you to manually call one of the `.to_*` methods to
convert a query to an in-memory format.

Depending on the library, this may work through a few mechanisms:

- Using the `__array__` protocol. Libraries expecting `numpy` arrays often also
  accept objects implementing this method.
- Using the `__dataframe__` protocol. This is a newer protocol, but is accepted
  by some libraries (a popular one is the plotting library `altair`)
- Using the `__arrow_c_stream__` protocol. This is also a new protocol for
  tooling passing around arrow tables.

---

For example, here we demonstrate passing an `ibis.Table` to the `altair` plotting
library. Notice that you don't have to manually call any of the `to_*` methods
yourself! This is because `altair` supports anything that implements the `__dataframe__`
protocol as input, which means it can accept an `ibis.Table` directly.

In [ ]:
import altair as alt

In [ ]:
# Make a scatterplot of bill length x bill depth for each penguin species
chart = (
    alt.Chart(penguins)  # <- here we pass in an `ibis.Table` directly
        .mark_circle(size=60)
        .encode(
            x=alt.X('bill_length_mm', scale=alt.Scale(zero=False)),
            y=alt.Y('bill_depth_mm', scale=alt.Scale(zero=False)),
            color=alt.Color('species'),
            tooltip=['species', 'sex', 'island']
        )
)

chart